# Preprocessing
- 공장별로 cond_24hr, cond_48hr 예측
- dataset은 공장데이터+asos(서산)+aws(당진,신평)+pred(송악읍)+date(시간 columns)

In [53]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

th_col=["tem_in_loc_24hr", "tem_in_loc_48hr", "hum_in_loc_24hr", "hum_in_loc_48hr", "tem_coil_loc_24hr", "tem_coil_loc_48hr"]
cond_col=[f"cond_loc{loc}", "cond_24hr", "cond_48hr"]

asos=pd.read_csv("train_data/plant_asos.csv",encoding='utf-8')
aws1=pd.read_csv("train_data/plant_616.csv",encoding='utf-8')
aws2=pd.read_csv("train_data/plant_637.csv",encoding='utf-8')
date=pd.read_csv("train_data/plant1_date.csv",encoding='utf-8')
pred=pd.read_csv("train_data/plant_pred.csv",encoding='utf-8')

model111 = RandomForestClassifier()
model112= RandomForestClassifier()
model113= RandomForestClassifier()
model121= RandomForestClassifier()
model122= RandomForestClassifier()
model123= RandomForestClassifier()
model211= RandomForestClassifier()
model212= RandomForestClassifier()
model213= RandomForestClassifier()
model221= RandomForestClassifier()
model222= RandomForestClassifier()
model223= RandomForestClassifier()

model_24=[[model111,model112,model113],[model121,model122,model123]]
model_48=[[model211,model212,model213],[model221,model222,model223]]

In [58]:
# 공백 전처리
l=[asos,aws2,date,pred]
for i in l:
    for j in i.columns:
        if i[j].isnull().sum()>30000:
            i.drop(j,axis=1,inplace=True)

## Model_fit
- 데이터의 비대칭이 심해서 smote활용

In [147]:
for plant in range(2,3):
    for loc in range(1,4):
        th_col=["tem_in_loc_24hr", "tem_in_loc_48hr", "hum_in_loc_24hr", "hum_in_loc_48hr", "tem_coil_loc_24hr", "tem_coil_loc_48hr"]
        cond_col=[f"cond_loc{loc}", "cond_24hr", "cond_48hr"]
        main=pd.read_csv(f"train/plant{plant}{loc}_train.csv",encoding='utf-8')
        for i in l:
            main=main.merge(i)
        main=main.loc[~(main.cond_24hr.isnull()|main.cond_48hr.isnull())]
        main.reset_index(drop=True,inplace=True)
        X=main.loc[:,col]
        y1=main.cond_24hr
        y2=main.cond_48hr
        
        X.drop('mea_ddhr',axis=1,inplace=True)
        
        X=X.interpolate(method='values')
        
        sm1=SMOTE()
        sm2=SMOTE()
        rX1, ry1 = sm1.fit_sample(X,y1)
        rX2, ry2 = sm2.fit_sample(X,y2)
        
        model_24[plant-1][loc-1].fit(rX1,ry1)
        model_48[plant-1][loc-1].fit(rX2,ry2)
        print(plant,loc)

2 1
2 2
2 3


# 공장별로 model prediction 계산

In [193]:
kosa=pd.read_csv("plant_test_24_48_date.csv",encoding='utf-8',index_col=0)
kosa.fillna(0,inplace=True)
kosa.columns=['mea_ddhr', 'plant', 'loc',
       'tem_in', 'hum_in', 'tem_coil',
       'tem_out_loc1', 'hum_out_loc1', 'tem_24hr',
       'hum_24hr', 'rain_p_24hr', 'wind_s_24hr', 'wind_d_24hr', 'sky_24hr_1',
       'sky_24hr_2', 'sky_24hr_3', 'sky_24hr_4', 'rain_t_24hr_0',
       'rain_t_24hr_1', 'rain_t_24hr_2', 'rain_t_24hr_3', 'rain_t_24hr_4',
       'tem_48hr', 'hum_48hr', 'rain_p_48hr', 'wind_s_48hr', 'wind_d_48hr',
       'sky_48hr_1', 'sky_48hr_2', 'sky_48hr_3', 'sky_48hr_4', 'rain_t_48hr_0',
       'rain_t_48hr_1', 'rain_t_48hr_2', 'rain_t_48hr_3', 'rain_t_48hr_4',
       'year', 'month', 'day', 'hour', 'week_1', 'week_2', 'week_3', 'week_4',
       'week_5', 'week_6', 'week_7', 'week_8', 'week_9', 'week_10', 'week_11',
       'week_12', 'week_13', 'week_14', 'week_15', 'week_16', 'week_17',
       'week_18', 'week_19', 'week_20', 'week_21', 'week_22', 'week_23',
       'week_24', 'week_25', 'week_26', 'week_27', 'week_28', 'week_29',
       'week_30', 'week_31', 'week_32', 'week_33', 'week_34', 'week_35',
       'week_36', 'week_37', 'week_38', 'week_39', 'week_40', 'week_41',
       'week_42', 'week_43', 'week_44', 'week_45', 'week_46', 'week_47',
       'week_48', 'week_49', 'week_50', 'week_51', 'week_52']
X=kosa.drop(["mea_ddhr","plant","loc","rain_t_24hr_4","rain_t_48hr_4"],axis=1)
cond_24=[]
prob_24=[]
cond_48=[]
prob_48=[]
for i in range(len(kosa)):
    p=kosa.plant[i]
    l=kosa["loc"][i]
    cond_24.append(model_24[int(p)-1][int(l)-1].predict(X.iloc[i].to_numpy().reshape(1,-1)))
    prob_24.append(model_24[int(p)-1][int(l)-1].predict_proba(X.iloc[i].to_numpy().reshape(1,-1))[0])
    cond_48.append(model_48[int(p)-1][int(l)-1].predict(X.iloc[i].to_numpy().reshape(1,-1)))
    prob_48.append(model_48[int(p)-1][int(l)-1].predict_proba(X.iloc[i].to_numpy().reshape(1,-1))[0])

In [202]:
c24=[]
p24=[]
c48=[]
p48=[]
for i in cond_24:
    c24.append(int(i[0]))
for i in cond_48:
    c48.append(int(i[0]))
for i in prob_24:
    p24.append(i[0]*100)
for i in prob_48:
    p48.append(i[0]*100)


# 답넣기

In [166]:
kosa

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc1,hum_out_loc1,tem_24hr,hum_24hr,rain_p_24hr,wind_s_24hr,wind_d_24hr,sky_24hr_1,sky_24hr_2,sky_24hr_3,sky_24hr_4,rain_t_24hr_0,rain_t_24hr_1,rain_t_24hr_2,rain_t_24hr_3,rain_t_24hr_4,tem_48hr,hum_48hr,rain_p_48hr,wind_s_48hr,wind_d_48hr,sky_48hr_1,sky_48hr_2,sky_48hr_3,sky_48hr_4,rain_t_48hr_0,rain_t_48hr_1,rain_t_48hr_2,rain_t_48hr_3,rain_t_48hr_4,year,month,day,hour,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14,week_15,week_16,week_17,week_18,week_19,week_20,week_21,week_22,week_23,week_24,week_25,week_26,week_27,week_28,week_29,week_30,week_31,week_32,week_33,week_34,week_35,week_36,week_37,week_38,week_39,week_40,week_41,week_42,week_43,week_44,week_45,week_46,week_47,week_48,week_49,week_50,week_51,week_52
0,2019-04-01 00:00:00,2,3,8.17,40.42,10.10,4.20,54.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2019-04-01 04:30:00,1,2,10.03,48.81,10.79,6.09,59.34,10.5,27.500000,0.0,4.650000,319.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.500000,30.000000,0.0,5.000000,310.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2019,4,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2019-04-01 10:30:00,2,3,9.45,40.93,9.07,12.26,32.14,7.0,41.666667,20.0,3.266667,280.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6.666667,43.333333,0.0,4.266667,265.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2019,4,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2019-04-01 11:00:00,1,1,12.32,34.13,10.86,12.32,28.27,6.0,48.333333,20.0,3.533333,275.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,5.333333,51.666667,0.0,3.933333,256.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2019,4,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2019-04-01 13:30:00,2,1,11.28,22.65,8.76,12.01,19.84,4.5,62.500000,15.0,3.450000,267.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.500000,67.500000,0.0,3.750000,237.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2019,4,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 03:00:00,1,2,10.73,33.36,13.30,6.28,46.37,12.0,35.000000,0.0,0.900000,281.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14.000000,40.000000,0.0,0.400000,332.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2020,3,29,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3535,2020-03-29 05:00:00,2,3,9.15,38.86,11.70,1.39,74.33,13.0,35.000000,0.0,1.650000,291.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14.500000,40.000000,0.0,1.050000,327.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2020,3,29,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3536,2020-03-29 07:00:00,2,1,7.10,44.87,9.03,4.26,58.06,12.0,40.000000,0.0,2.650000,307.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.500000,40.000000,0.0,2.250000,328.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2020,3,29,7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3537,2020-03-29 11:30:00,1,1,13.54,33.49,12.20,12.90,33.67,8.0,57.500000,0.0,2.150000,324.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.000000,50.000000,10.0,2.250000,334.5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2020,3,29,11,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
answer=pd.read_csv("before_result.csv",encoding='utf-8')

In [77]:
answer

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,0,100,2019-04-03 0:00,0,100
1,2019-04-01 4:30,1,2,2019-04-02 4:30,0,100,2019-04-03 4:30,0,100
2,2019-04-01 10:30,2,3,2019-04-02 10:30,0,100,2019-04-03 10:30,0,100
3,2019-04-01 11:00,1,1,2019-04-02 11:00,0,100,2019-04-03 11:00,0,100
4,2019-04-01 13:30,2,1,2019-04-02 13:30,0,100,2019-04-03 13:30,0,100
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,0,100,2020-03-31 3:00,0,100
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,0,100,2020-03-31 5:00,0,100
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,0,100,2020-03-31 7:00,0,96
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,0,100,2020-03-31 11:30,0,100


In [203]:
answer["X24H_COND_LOC"]=c24
answer["X24H_COND_LOC_PROB"]=p24
answer["X48H_COND_LOC"]=c48
answer["X48H_COND_LOC_PROB"]=p48

answer.to_csv("203756.csv",encoding="utf-8",index=False)

In [204]:
answer

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,0,99.0,2019-04-03 0:00,0,98.0
1,2019-04-01 4:30,1,2,2019-04-02 4:30,0,94.0,2019-04-03 4:30,0,94.0
2,2019-04-01 10:30,2,3,2019-04-02 10:30,0,100.0,2019-04-03 10:30,0,100.0
3,2019-04-01 11:00,1,1,2019-04-02 11:00,0,99.0,2019-04-03 11:00,0,100.0
4,2019-04-01 13:30,2,1,2019-04-02 13:30,0,96.0,2019-04-03 13:30,0,100.0
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,0,100.0,2020-03-31 3:00,0,98.0
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,0,100.0,2020-03-31 5:00,0,100.0
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,0,100.0,2020-03-31 7:00,0,100.0
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,0,100.0,2020-03-31 11:30,0,100.0
